In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from keras import backend as K

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pickle
import h5py
import cv2
import os
import glob
import time

from keras.layers import merge
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Reshape, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import Adadelta, Adam, SGD
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.callbacks import EarlyStopping, History, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd

from collections import Counter

from keras.models import load_model
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from keras.applications.xception import Xception
from keras.models import Model
from keras.utils.np_utils import to_categorical
import shutil

from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans

import numpy as np
from various_utils_general import *

Using TensorFlow backend.
/home/w/anaconda3/envs/idp3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
srcvgg = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Intel_Cervix/data/test_crops_vgg/'
dstvgg = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Intel_Cervix/data/training_data/test_crops_vgg_299/test_crops_vgg_299/'
resize_test(srcvgg, dstvgg)

Read test images
Resized test data time: 65.24 seconds


In [ ]:
def prep_test(te_path):
    t = time.time()
    X_test, test_ids = load_test(te_path)
    X_test2 = X_test.astype('float32')
    X_test2 = preprocess_input(X_test2)
    features = model.predict(X_test2)
    features = features.reshape(len(X_test2), -1)
    print('Reshaped: ', features.shape)
    print('Time it took to prepare test data:', time.time() - t)
    return X_test2, test_ids, features

def prep_train(tr_path):
    t = time.time()
    X_train, y_train, train_ids = load_train(tr_path)
    X_train2 = X_train.astype('float32')
    X_train2 = preprocess_input(X_train2)
    features = model.predict(X_train2, batch_size = 16)
    features = features.reshape(len(X_train2), -1)
    print('Reshaped: ', features.shape)
    print('Time it took to prepare test data:', time.time() - t)
    return X_train, y_train, train_ids, features


def fit_knn(neighs, features):
    knn = NearestNeighbors(n_neighbors = neighs, n_jobs = 6, metric = 'hamming', algorithm = 'brute')
    knn.fit(features)
    neighbors = knn.kneighbors(features)
    return neighbors

def normalize_probs(df):
    df2 = df.copy().iloc[:, :-1]
    df3 = df.copy().iloc[:, :-1]
    for i in df2.columns:
        df3[i] = df2[i] / df2.sum(axis = 1)
    return df3

def get_averaged(neighbors):
    neigh = fit_knn(neighbors, features_test)
    sub_knn = nn_probs(neigh)
    sub_knn = normalize_probs(sub_knn)
    sub_knn['image'] = sub1['image']
    return sub_knn


def get_similar(neighbors_list, threshold, savename):
    imgs = []
    for i in range(len(neighbors_list[0])):
        for j in range(1, neighbors_list[0].shape[1]):
            if abs(neighbors_list[0][i][0] - neighbors_list[0][i][j]) <= threshold and abs(neighbors_list[0][i][0] - neighbors_list[0][i][j]) >= 0.0001:
                print('Image index:', neighbors_list[1][i][j])
                #print('Distances:', neighbors_list[0][i])
                #print('Neighbors:', neighbors_list[1][i])
                imgs.append(neighbors_list[1][i][j])
    imgs = list(set(imgs))
    imgs_todel = []
    for i in imgs:
        imgs_todel.append(train_ids[i])
        with open('{}.txt'.format(savename), 'a') as out:
            out.write(train_ids[i] + '\n')
    return imgs_todel

def delete_similar(src, imgstodel):
    folders = ['Type_1', 'Type_2', 'Type_3']
    dst = src + 'to_delete/'
    if 'to_delete/' not in os.listdir(src):
        os.mkdir(src + 'to_delete/')
    for fld in folders:
        index = folders.index(fld)
        print('Load folder {} (Index: {})'.format(fld, index))
        path = os.path.join(src, fld, '*.jpg')
        files = glob.glob(path)
        for todel in imgstodel:
            for fl in files:
                if todel in fl:
                    flbase = os.path.basename(fl)
                    shutil.move(fl, dst + flbase)
    return
               
def show_neighbors(X, neighbors, index, index2):
    print(neighbors[0][index])
    print(neighbors[1][index])
    fig, ax = plt.subplots(1,2, figsize = (20, 12))
    ax[0].imshow(X[index])
    ax[1].imshow(X[index2])
    return
    
tr_path = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Intel_Cervix/data/train/'
te_path = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Intel_Cervix/data/test/'
tradd_path = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Intel_Cervix/data/train_additional/'

base_model = VGG19(weights='imagenet', include_top = False, input_shape = (299, 299, 3))
model = Model(input=base_model.input, output=base_model.get_layer('block5_pool').output)

In [ ]:
def resize_train(src, dst):
    start_time = time.time()
    print('Read train images')
    folders = ['Type_1', 'Type_2', 'Type_3']
    for fld in folders:
        index = folders.index(fld)
        print('Load folder {} (Index: {})'.format(fld, index))
        path = os.path.join(src, fld, '*.jpg')
        files = glob.glob(path)
        for i, fl in enumerate(files):
            flbase = os.path.basename(fl)
            flbase = '{}/id{}_vgg11_{}'.format(fld, flbase.split('.')[0], i) + '.jpg'
            try:
                img = get_im_cv2(fl)
            except Exception:
                print('Failed for image:', fl)
                continue
            res_img = cv2.resize(img, size, cv2.INTER_AREA)
            final = Image.fromarray((res_img).astype(np.uint8))
            final.save(dst + flbase)
    print('Resized train data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return

In [ ]:
dst_te = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Intel_Cervix/data/test_crops_vgg_299/'
resize_test(te_path, dst_te)

In [ ]:
tr_path = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Intel_Cervix/data/train_crops_vgg11/'
dst = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Intel_Cervix/data/train_crops_vgg11_299/'
resize_train(tr_path, dst)

In [ ]:
X_train, y_train, train_ids, features = prep_train(tr_path)
np.save('raw_original_resized', X_train)
np.save('raw_original_features', features)
np.save('raw_original_ids', train_ids)

In [ ]:
X_test, test_ids, features = prep_test(te_path)
np.save('raw_test_resized', X_train)
np.save('raw_test_features', features)
np.save('raw_test_ids', test_ids)

In [ ]:
def fit_knn(neighs, features):
    knn = NearestNeighbors(n_neighbors = neighs, n_jobs = 6, metric = 'chebyshev', algorithm = 'brute')
    knn.fit(features)
    neighbors = knn.kneighbors(features)
    return neighbors

In [ ]:
#n3 = fit_knn(3, features)
#n5 = fit_knn(5, features)
n8 = fit_knn(8, features_train_add)
#n8te = fit_knn(8, features_test)

Correlation, thres = 3e-01, maybe 4e-01 seems to give good results

In [ ]:
imgstodel_trorig = get_similar(n8, 3e-01, 'additonaltrain_raw_0.3_correlationdist')
#delete_similar(tr_path, imgstodel_trorig)

In [ ]:
imgstodel_teorig = get_similar(n8te, 0.4, 'similar_original_test_0.4')
#delete_similar(te_path, imgstodel_teorig)